In [4]:
import sys
import os
sys.path.append('../')
os.chdir('../')

import pandas as pd
import numpy as np
import shutil
from pathlib import Path
from datetime import datetime
import time
import threading
import json
import requests
import threading  
from src import utils  

Posibles tesauros:

    - MedDRA (Diccionario Médico para Actividades Reguladoras)
    - DeCS/MeSH

In [5]:
info = "data/prueba"

In [6]:
def read_medicamento_info(filename):
    with open(filename,"r") as file:
        data = json.load(file)
    return data

In [7]:
all_data = []
for element in os.listdir(info):
    nregistro = int(element[:-5])
    print(nregistro)
    data = read_medicamento_info(os.path.join(info,element))
    all_data.append(data)
    print(data)
    break

12723
[{'seccion': '1', 'titulo': 'NOMBRE DEL MEDICAMENTO', 'contenido': '<div>\r\n    <p style="margin:0pt; line-height:115%; font-size:11pt"><span style="font-family:Times New Roman; font-size:11pt">Antabus 250 mg comprimidos</span></p>\r\n  </div>', 'orden': 1}, {'seccion': '2', 'titulo': 'COMPOSICIÓN CUALITATIVA Y CUANTITATIVA', 'contenido': '<div>\r\n    <p style="margin:0pt"><span style="font-family:Times New Roman; font-size:11pt">Cada comprimido contiene 250 mg de disulfiram.</span></p>\r\n\r\n    <p style="margin:0pt"><span style="font-family:Times New Roman; font-size:11pt">&#xa0;</span></p>\r\n\r\n    <p style="margin:0pt"><span style="font-family:Times New Roman; font-size:11pt">Para consultar la lista completa de excipientes ver secci&#243;n 6.1.</span></p>\r\n\r\n    <p style="margin:0pt; line-height:115%; font-size:11pt"><span style="font-family:Times New Roman; font-size:11pt">&#xa0;</span></p>\r\n  </div>', 'orden': 1}, {'seccion': '3', 'titulo': 'FORMA FARMACÉUTICA', 

Lectura de todos los medicamentos que tenemos en la carpeta info, en este caso se enseña el nregistro del medicamento, 

In [8]:
name_original = utils.extract_and_clean_text(data[0]["contenido"])
name = utils.extract_text_before_number(name_original)
text = utils.extract_and_clean_text(data[14]["contenido"])
print(name_original,", ",name)

Antabus 250 mg comprimidos ,  Antabus


In [9]:
def bar(text):
    from IPython.display import display, HTML
    chart = HTML(text)
    # or chart = charts.plot(...)
    display(chart)
    return chart

chart = bar(data[14]["contenido"])

In [10]:
## Este metodo se encarga de quitar las etiquetas del html y eliminar tabs

# TODO:
#   - Mirar si funciona con todos los tipos de punto 4.8

from bs4 import BeautifulSoup
import re

html_content = data[14]["contenido"]
# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Extract text and filter out empty lines
text = soup.get_text(separator='\n').strip()
cleaned_text = re.sub(r'\xa0', '', text)

lines = re.split(r'\n{2,}', cleaned_text)
for index,element in enumerate(lines):
    cleaned_text = re.sub(r'\n', '', element)
    lines[index] = cleaned_text

cleaned_text = '\n'.join(lines)

In [11]:
lines

['El disulfiram es, por sí mismo, una sustancia relativamente no tóxica, observándose escasos efectos secundarios cuando se administra sólo a dosis terapéuticas en animales y seres humanos.',
 'Los efectos secundarios durante la reacción disulfiram-alcohol se han descrito en la Sección 4.4.',
 'La mayoría de las reacciones al disulfiram son autolimitadas y no representan un riesgo vital para el paciente.',
 'Un número reducido de pacientes pueden experimentar adormeciendo leve transitorio, fatiga, impotencia, cefaleas, erupciones acneiformes, dermatitis alérgica, o sabor metálico o a ajo durante las dos primeras semanas de tratamiento. Estos síntomas suelen desaparecer espontáneamente con la continuación del tratamiento o la reducción de la posología.',
 'Reacciones adversas ordenadas según frecuencia de aparición:',
 'Trastornos psiquiátricos:',
 'Muy raras (<1/10.000): Episodios psicóticos, incluyendo, paranoia, esquizofrenia..',
 'Frecuentes (≥1/100 a <1/10): depresión, manía.',
 'T

In [13]:
# Remove capital letters and periods
low_not_dots = ''.join([char for char in cleaned_text.lower() if char != '.'])
low_not_dots


'el disulfiram es, por sí mismo, una sustancia relativamente no tóxica, observándose escasos efectos secundarios cuando se administra sólo a dosis terapéuticas en animales y seres humanos\nlos efectos secundarios durante la reacción disulfiram-alcohol se han descrito en la sección 44\nla mayoría de las reacciones al disulfiram son autolimitadas y no representan un riesgo vital para el paciente\nun número reducido de pacientes pueden experimentar adormeciendo leve transitorio, fatiga, impotencia, cefaleas, erupciones acneiformes, dermatitis alérgica, o sabor metálico o a ajo durante las dos primeras semanas de tratamiento estos síntomas suelen desaparecer espontáneamente con la continuación del tratamiento o la reducción de la posología\nreacciones adversas ordenadas según frecuencia de aparición:\ntrastornos psiquiátricos:\nmuy raras (<1/10000): episodios psicóticos, incluyendo, paranoia, esquizofrenia\nfrecuentes (≥1/100 a <1/10): depresión, manía\ntrastornos del sistema nervioso:\nfr

In [21]:
a = low_not_dots.split("\n")

In [24]:
b = re.sub(':', '', low_not_dots)
c = b.split("\n")

In [3]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Supongamos que este es nuestro tesauro simplificado
tesauro = {
    "corazón": ["cardíaco", "miocardio"],
    "dolor": ["molestia", "incomodidad"]
}

# Función para buscar sinónimos en el tesauro
def buscar_sinonimos(palabra, tesauro):
    for key, sinonimos in tesauro.items():
        if palabra in sinonimos:
            return key
    return palabra

# Texto de ejemplo
texto = "El paciente experimenta molestia en el área cardíaca."

# Tokenización del texto
tokens = word_tokenize(texto)

# Reemplazo de sinónimos
texto_procesado = [buscar_sinonimos(palabra, tesauro) for palabra in tokens]

# Reconstruir el texto
texto_reconstruido = ' '.join(texto_procesado)
print("Texto Original:", texto)
print("Texto Procesado:", texto_reconstruido)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Texto Original: El paciente experimenta molestia en el área cardíaca.
Texto Procesado: El paciente experimenta dolor en el área cardíaca .


El texto a analizar debe ser procesado adecuadamente. Esto implica limpiar el texto (eliminar ruido, normalizar términos, etc.), 
tokenizarlo (dividirlo en palabras o frases individuales) y posiblemente lematizarlo (reducir las palabras a su forma base).

pip install nltk
pip install spacy
python -m spacy download es_core_news_sm # Descargar el modelo en español para spaCy

In [38]:
import spacy
import re
from nltk.tokenize import word_tokenize

# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')

# Texto de ejemplo
texto = c

# Función para limpiar el texto
def limpiar_texto(texto):
    # Eliminar caracteres especiales y números
    texto = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]', '', texto)
    # Convertir a minúsculas
    return texto.lower()

# Lematizar el texto
def lematizar_texto(texto):
    doc = nlp(texto)
    return ' '.join([token.lemma_ for token in doc])

# Procesar cada línea del texto
new_texto = []
for linea in texto:
    linea_limpia = limpiar_texto(linea)
    linea_lematizada = lematizar_texto(linea_limpia)
    tokens = word_tokenize(linea_lematizada)
    new_texto.append(tokens)
    # print("Original:", linea)
    # print("Limpio:", linea_limpia)
    # print("Lematizado:", linea_lematizada)
    print("Tokens:", tokens)
    # print("\n")
# print(new_texto)

Tokens: ['el', 'disulfiram', 'ser', 'por', 'él', 'mismo', 'uno', 'sustancia', 'relativamente', 'no', 'tóxico', 'observar', 'él', 'escaso', 'efecto', 'secundario', 'cuando', 'él', 'administrar', 'sólo', 'a', 'dosis', 'terapéutico', 'en', 'animal', 'y', 'ser', 'humano']
Tokens: ['el', 'efecto', 'secundario', 'durante', 'el', 'reacción', 'disulfiramalcohol', 'él', 'haber', 'describir', 'en', 'el', 'sección']
Tokens: ['el', 'mayoría', 'de', 'el', 'reacción', 'al', 'disulfiram', 'ser', 'autolimitada', 'y', 'no', 'representar', 'uno', 'riesgo', 'vital', 'para', 'el', 'paciente']
Tokens: ['uno', 'número', 'reducido', 'de', 'paciente', 'poder', 'experimentar', 'adormecer', 'leve', 'transitorio', 'fatigo', 'impotencia', 'cefalea', 'erupción', 'acneiform', 'dermatitis', 'alérgico', 'o', 'sabor', 'metálico', 'o', 'a', 'ajo', 'durante', 'el', 'dos', 'primero', 'semana', 'de', 'tratamiento', 'este', 'síntoma', 'soler', 'desaparecer', 'espontáneamente', 'con', 'el', 'continuación', 'del', 'tratamien

In [44]:
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')

# Lista de stop words en español
stop_words = set(stopwords.words('spanish'))

listas_filtradas = [[elemento for elemento in lista if elemento not in stop_words] for lista in new_texto]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
for element in listas_filtradas:
    print(element)

['disulfiram', 'ser', 'mismo', 'sustancia', 'relativamente', 'tóxico', 'observar', 'escaso', 'efecto', 'secundario', 'administrar', 'sólo', 'dosis', 'terapéutico', 'animal', 'ser', 'humano']
['efecto', 'secundario', 'reacción', 'disulfiramalcohol', 'haber', 'describir', 'sección']
['mayoría', 'reacción', 'disulfiram', 'ser', 'autolimitada', 'representar', 'riesgo', 'vital', 'paciente']
['número', 'reducido', 'paciente', 'poder', 'experimentar', 'adormecer', 'leve', 'transitorio', 'fatigo', 'impotencia', 'cefalea', 'erupción', 'acneiform', 'dermatitis', 'alérgico', 'sabor', 'metálico', 'ajo', 'dos', 'primero', 'semana', 'tratamiento', 'síntoma', 'soler', 'desaparecer', 'espontáneamente', 'continuación', 'tratamiento', 'reducción', 'posología']
['reacción', 'adverso', 'ordenado', 'según', 'frecuencia', 'aparición']
['trastorno', 'psiquiátrico']
['rara', 'episodio', 'psicótico', 'incluir', 'paranoia', 'esquizofrenia']
['frecuente', 'depresión', 'manir']
['trastorno', 'sistema', 'nervioso'